In [37]:
import os
import pprint
from pymongo import MongoClient

filepath = "C:/Users/Saad Khalid/Downloads/islamabad_pakistan.osm"

In [38]:
run shape_data.py

In [39]:
## ===Takes time to process===
data = process_map(filepath, True)
## ===Now Upload the json file onto the database===

In [40]:
print "Nunmber of documents:",len(data)

Nunmber of documents: 344057


In [41]:
## to print the desired number of documents from a pymongo cusrsor obtained through querying
## entries = 0 or entries = "all", the function prints all documents
def print_aggregation(aggregation, entries):  
    if entries == 0 or entries == "all":
        for i in aggregation:
            pprint.pprint(i)
    elif entries > 0:
        count = 0
        for i in aggregation:
            pprint.pprint(i)
            count += 1
            if count == entries:
                break
    else:
        print "Invalid entries number"

In [45]:
## finding most contributions by users
client = MongoClient('localhost:27017')
db = client.cities

user = db.islamabad.aggregate([{"$group" : {"_id" : "$created.user", "count" : {"$sum" : 1}}},
                               {"$sort" : {"count" : -1}}])

print_aggregation(user,5)

{u'_id': u'girgit', u'count': 57825}
{u'_id': u'MMDawood', u'count': 41203}
{u'_id': u'dcguj', u'count': 26631}
{u'_id': u'shaji', u'count': 23391}
{u'_id': u'Mateen Bhatti', u'count': 22802}


In [46]:
## Listing the top amenities:
amenities = db.islamabad.aggregate([{"$group" : {"_id" : "$amenity", "count" : {"$sum" : 1}}},
                               {"$sort" : {"count" : -1}}])
print_aggregation(amenities,5)

{u'_id': None, u'count': 341480}
{u'_id': u'fuel', u'count': 498}
{u'_id': u'place_of_worship', u'count': 354}
{u'_id': u'school', u'count': 311}
{u'_id': u'parking', u'count': 243}


In [47]:
## Listing a few cafes found in Islamabad
cafes = db.islamabad.find({"amenity":"cafe"}, {"name" : True})
print_aggregation(cafes,5)

{u'_id': ObjectId('5547d7badeaeaa0452cc85f1'), u'name': u'The Hot Spot'}
{u'_id': ObjectId('5547d7badeaeaa0452cc85f3'), u'name': u'The Gelato Affair'}
{u'_id': ObjectId('5547d7badeaeaa0452cc85f4'), u'name': u'The Hot Spot Cafe'}
{u'_id': ObjectId('5547d7badeaeaa0452cc85fe'), u'name': u'Civil Junction'}
{u'_id': ObjectId('5547d7badeaeaa0452ccc772'),
 u'name': u"Officer's Club (Marriage Hall)"}


In [48]:
## Listing a few schools found in Islamabad
cafes = db.islamabad.find({"amenity":"school"}, {"name" : True})
print_aggregation(cafes,5)

{u'_id': ObjectId('5547d7b9deaeaa0452cc3c0b'),
 u'name': u'Islamabad College for Girls'}
{u'_id': ObjectId('5547d7b9deaeaa0452cc61b8'),
 u'name': u'FG Boys Model School'}
{u'_id': ObjectId('5547d7b9deaeaa0452cc7eeb')}
{u'_id': ObjectId('5547d7b9deaeaa0452cc7fc0'),
 u'name': u'FG Junior Model School'}
{u'_id': ObjectId('5547d7b9deaeaa0452cc7fc1'),
 u'name': u'F.G Model School No. 36 G-9/2'}


In [49]:
phones = db.islamabad.find({"phone" : {"$exists" : 1}}, {"phone" : 1})
print_aggregation(phones,0)

{u'_id': ObjectId('5547d7bcdeaeaa0452cd685f'), u'phone': u'+92-51-2851267'}
{u'_id': ObjectId('5547d7bcdeaeaa0452cd68c7'), u'phone': u'+92-51-8319000'}
{u'_id': ObjectId('5547d7bcdeaeaa0452cd74ec'), u'phone': u'+92-51-2852024'}
{u'_id': ObjectId('5547d7bddeaeaa0452cdee93'), u'phone': u'+92-51-2872639'}
{u'_id': ObjectId('5547d7bddeaeaa0452cdefd1'),
 u'phone': [u'+92-51-4438559', u'+92-51-5839323']}
{u'_id': ObjectId('5547d7bddeaeaa0452cdf5d4'), u'phone': u'+92-300-5205536'}
{u'_id': ObjectId('5547d7bddeaeaa0452cdfc32'), u'phone': u'+92-51-2820304'}
{u'_id': ObjectId('5547d7bddeaeaa0452cdffcb'), u'phone': u'+92-51-2650953'}
{u'_id': ObjectId('5547d7bddeaeaa0452cdffce'), u'phone': u'+92-51-2609301'}
{u'_id': ObjectId('5547d7bddeaeaa0452cdffd1'), u'phone': u'+92-51-8445544'}
{u'_id': ObjectId('5547d7bddeaeaa0452ce2304'), u'phone': u'+92-51-111901901'}
{u'_id': ObjectId('5547d7bddeaeaa0452ce2531'), u'phone': u'+92-51-2277117'}
{u'_id': ObjectId('5547d7bddeaeaa0452ce6ee0'), u'phone': u'+92-

In [50]:
## listing unique postcodes in dataset, the postcodes of Rawalpindi(twin city of Islamabad), Attock and Abbotabad 
## are also present
test = db.islamabad.aggregate([{"$match" : {"address.postcode": {"$exists" : 1}}},
                              {"$group" : {"_id" : "postcode", "codes" : {"$addToSet" : "$address.postcode"}, "count" : {"$sum": 1}}}])
print_aggregation(test,0)

{u'_id': u'postcode',
 u'codes': [u'46600',
            u'46060',
            u'4600',
            u'10530',
            u'22010',
            u'46000',
            u'44000'],
 u'count': 60}


In [51]:
selection = db.islamabad.find_one({"address.postcode" : "4600"})
selection["address"]["postcode"] = "46000"
db.islamabad.save(selection)


In [ ]:
selectiodb.islamabad.find_one({"address.postcode" : "4600"})

In [55]:
street_add=db.islamabad.aggregate([{"$match" : {"address.street" : {"$exists" : 1}}},
                           {"$group" : {"_id" : "addresses", "addr" : {"$addToSet" : "$address.street"}}}])
print_aggregation(street_add,0)

{u'_id': u'addresses',
 u'addr': [u'Street 1',
           u'Cantt',
           u'Islamabad Express Way',
           u'Eastridge Road',
           u'Siddiq-e-Akbar Street',
           u'Tayyeba Street',
           u'Street 17',
           u'Circular Road',
           u'Main Bazar Road',
           u'Cantonment Plaza Road',
           u'Club Road',
           u'Street 5 Iqbal Road',
           u'Nazim ud Din Road',
           u'Police Station Street',
           u'Devi Data Road',
           u'Commerce College Road',
           u'Sukho Gujar Khan Road',
           u'Lane 8 A, Sector H',
           u'Johar Road',
           u'Asghar Mall Road',
           u'Phimbal Road',
           u'Park Road',
           u'Muncipal Road',
           u'Housing Scheme No.1',
           u'GT Road',
           u'Link',
           u'Street 98',
           u'Street 78',
           u'Club Avenue',
           u'Murree Road',
           u'Street 13, Dhoke Kashmirian',
           u'Lane-1',
           u'Street 2

In [56]:
## counting postcode occurence
db.islamabad.find({"address.postcode" : "10530"}).count()

3

In [57]:
unique_users = db.islamabad.aggregate([{"$group" : {"_id" : "user", "user" : {"$addToSet" : "$created.user"}}},
                                      {"$unwind" : "$user"},
                                      {"$group" : {"_id" : "user", "count" : {"$sum" : 1}}}])
print_aggregation(unique_users,0)


{u'_id': u'user', u'count': 356}


In [58]:
db.islamabad.find({"type" : "node"}).count()

314514

In [59]:
db.islamabad.find({"type" : "way"}).count()

29540

In [60]:
db.islamabad.find({"amenity" : "cafe"}).count()

27

In [61]:
db.islamabad.find({"amenity" : "school"}).count()

311

In [62]:
db.islamabad.find({"address.postcode" : "4600"}).count()

0

In [35]:
db.islamabad.drop()